# Settings

In [ ]:
import requests
import datetime 
import re
import lxml.html as html
import pandas as pd
import os
import time
import json
import numpy as np


pd.options.display.float_format='{:,.1f}'.format
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#EL proyecto se ejecuto en GoogleColab, en caso de querer correr el proyecto en un entorno local se deberá definir el directorio de trabajo con el módulo os
%cd '/content/drive/My Drive/Notebooks Dataset/'
!ls
!pwd

/content/drive/My Drive/Notebooks Dataset
/content/drive/My Drive/Notebooks Dataset


# Functions

In [ ]:
def extract_n_pages(url):
    '''
    Mediante una consulta escrapea la cantidad de paginas en una categoria, retorna la cantidad de paginas como int
    '''
    response = requests.get(url)
    if response.status_code == 200: 
                home = response.content.decode('utf-8') 
                parsed = html.fromstring(home)
    else:
        raise ValueError(f'Error, no se pudo obtener la cantidad de paginas: {response.status_code} ')
    
    xpath_expresion = '//ul[@class="pagination"]//a[@role="button"]/text()'
    last_page_code = int(parsed.xpath(xpath_expresion)[-2])
    
    print('Hay que extraer info de {} paginas'.format(last_page_code))
    return last_page_code



In [ ]:
def create_urls(base_url):
    '''
    recibe el link base de la categoria y retorna una lista con todos los links de productos de esa categoria
    '''
    n_pages = extract_n_pages(base_url)
    iterable_link = base_url+'?ordering=offer_price_usd&page={}&'
    links_list = []
    for i in range(1, n_pages+1):
        links_list.append(iterable_link.format(i))
    
    print('Se han creado los {} links para comenzar el Scrapping'.format(n_pages))
    return links_list



In [ ]:
def clean_text(text_list):
    '''
    Recibe una lista con los textos scrapeados y los limpia usando regex, retorna una lista con los textos limpios
    '''
    clean_text = []
    for name in text_list:
        del_shit = re.sub('\r\n\t[\t]?','',name)
        if len(del_shit) > 4:
            clean_text.append(del_shit)
    return clean_text


In [ ]:

def extract_info(links_list):
    '''
    Recibe una lista con los links de una categoria y scrapea la informacion
    retorna listas con todos los specs
    '''
    pause = 6 #segundos de pausa entre request
    print('Se extá obteniendo la información broda, esto demorará {} segundos '.format(len(links_list) * pause))
    
    names = []
    prices = []
    processor = []
    ram = []
    screen = []
    storages = []
    graphics = []
    links = []
    ids = []
    Brand = []
    
    i = 1
    for link in links_list:
        time.sleep(pause)
        response = requests.get(link)
        if response.status_code == 200:
            home = response.content.decode('utf-8') 
            parsed = html.fromstring(home)
            
        else:
            raise ValueError(f'Error, VTR arruinó el scrapeo: {response.status_code} ')
        
        names_xpath = '//div[@class="d-flex flex-column category-browse-result"]/h3/a/text()'
        processors_xpath = '//div[@class="d-flex flex-column category-browse-result"]/div[@class="description-container"]/dl/dd[1]/text()'
        rams_xpath = '//div[@class="d-flex flex-column category-browse-result"]/div[@class="description-container"]/dl/dd[2]/text()'
        screens_xpath = '//div[@class="d-flex flex-column category-browse-result"]/div[@class="description-container"]/dl/dd[3]/text()'
        prices_xpath = '//div[@class="d-flex flex-column category-browse-result"]//div[@class="price flex-grow"]/a/text()'
        storage_xpath = 'dl/dd[4]/ul/li/text()'
        graphics_xpath = 'dl/dd[5]/ul/li/text()'
        url_xpath = '//div[@class="d-flex flex-column category-browse-result"]/div[@class="image-container d-flex flex-column justify-content-center"]/a/@href'
        BASE_ITEMS = '//div[@class="d-flex flex-column category-browse-result"]/div[@class="description-container"]'
        
        
        price_list = parsed.xpath(prices_xpath)
        for price in price_list:
            del_simbol = price.replace("$ ", "")
            del_dot = int(del_simbol.replace(".", ""))
            prices.append(del_dot)
            
        names.extend(parsed.xpath(names_xpath))
        ram.extend(parsed.xpath(rams_xpath))
        
        product_links = parsed.xpath(url_xpath)
        
        for link in product_links:
            id = link.replace("/products/","")
            brand_r = re.compile(r'^\d{1,6}-(\w{1,15})-')
            brand = brand_r.findall(id)
            brandf = brand[0].upper()
            Brand.append(brandf)
        
        for link in product_links:
            base_url = 'https://www.solotodo.cl'
            full_url = base_url+link
            links.append(full_url)
        
        
        for link in product_links:
            id = link.replace("/products/","")
            ids.append(id)
        
        dirt_processor = (parsed.xpath(processors_xpath))
        processor.extend(clean_text(dirt_processor))
        
        dirt_screen = (parsed.xpath(screens_xpath))
        screen.extend(clean_text(dirt_screen))
        
        
        items = parsed.xpath(BASE_ITEMS)
        for item in items:
            spec = item.xpath('dl/dd[5]/ul/li/text()')
            if len(spec)>1:
                specjoin = ''.join(spec)
                graphics.append(specjoin)
            else: graphics.extend(spec)
        
        for item in items:
            spec = item.xpath('dl/dd[4]/ul/li/text()')
            if len(spec)>1:
                specjoin = ''.join(spec)
                storages.append(specjoin)
            else: storages.extend(spec)
        
        #A diferencia de los otros specs, almacenamiento y gráfica puede retornar una lista de dos elementos
        #Ya que un note puede tener hdd y/o ssd, y tarjeta de video integrada y/o dedicada
        #así que necesitaron tratamiento especial
        
        print('Lista la hoja numero : {}'.format(i))
        i +=1
     
    print('Listo mi rey, se escrapeó la información de {} notebooks'.format(len(ids)))
    return names,prices,processor,ram,screen,storages,graphics,links,ids,Brand


In [ ]:
#Estan son todas las funciones para poder refinar los datos en bruto escrapeados

def details_ram(ram_list):
    '''
    Recibe la lista con los datos brutos de memoria RAM y lo separa de forma más detallada
    '''
    ram_gb = []
    DDR = []
    Mhz = []
    
    #Recuerda que findall devuelve una lista
    for m_ram in ram_list:
        ramGB = re.compile(r'^\d\d?')
        ram = ramGB.findall(m_ram)
        ram_gb.append(int(ram[0]))

        ddr = re.compile(r'DDR\d\w?')
        DDR.extend(ddr.findall(m_ram))

        mhz = re.compile(r'(\d{4}) MHz')
        Mhz.extend(mhz.findall(m_ram))
        
    return ram_gb,DDR,Mhz
    
    
def details_screen(screens_list):
    '''
    Recibe la lista con los datos brutos de pantalla y lo separa de forma más detallada
    '''
    inches = []
    resolution = []
    Hz = []
    
    #Recuerda que findall devuelve una lista
    for monitor in screens_list:
        inche = re.compile(r'\d{2}\.\d')
        inche_n = inche.findall(monitor)
        inches.append(float(inche_n[0]))
        
        
        res = re.compile(r'\d{3,4}x\d{3,4}')
        resolution.extend(res.findall(monitor))

        hz = re.compile(r'(\d{2,3}) Hz')
        Hz.extend(hz.findall(monitor))
        
    return inches,resolution,Hz

def details_storage(storage_list):
    '''
    Recibe la lista con los datos brutos de almacenamiento y lo separa de forma más detallada
    '''
    hard_disc = []
    solid_disc = []
    Optane = []
    eMMC = []
    
    #Recuerda que findall devuelve una lista
    for storage in storage_list:
        emmc_r = re.compile(r'eMMC (\d{2,3})[GB]+')
        emmc = emmc_r.findall(storage)
        if len(emmc) == 0:
            eMMC.append(0)
        else:
            eMMC.append(int(emmc[0]))

        hdd_r = re.compile(r'HDD (\d{1,3})[GB|TB]+')
        hdd = hdd_r.findall(storage)
        if len(hdd) == 0:
            hard_disc.append(0)
            
        if len(hdd) != 0 and int(hdd[0]) < 5:
            hard_disc.append(int(hdd[0])*1000)
            
        if len(hdd) != 0 and int(hdd[0]) > 5:
            hard_disc.append(int(hdd[0]))


        ssd_r = re.compile(r'SSD (\d{1,3})[GB|TB]+')
        ssd = ssd_r.findall(storage)
        if len(ssd) == 0:
            solid_disc.append(0)
        if len(ssd) != 0 and int(ssd[0]) < 5:
            solid_disc.append(int(ssd[0])*1000)
            
        if len(ssd) != 0 and int(ssd[0]) > 5:
            solid_disc.append(int(ssd[0]))


        optane_r = re.compile(r'Intel Optane (\d{1,3})[GB]+')
        optane = optane_r.findall(storage)
        if len(optane) == 0:
            Optane.append(0)
        else:
            Optane.append(int(optane[0]))
    
    return eMMC,hard_disc,solid_disc,Optane

def details_processor(processor_list):
    '''
    Recibe la lista en bruto con la info de los procesadores y retorna listas con los datos separados.
    '''
    processor_brand = []
    processor_model = []
    cores = []
    threads = []
    processor_mhz = []
    
    for processor in processor_list:
            brand_r = re.compile(r'^(\w{1,6}) ')
            p_brand = brand_r.findall(processor)
            processor_brand.extend(p_brand)

            model_r = re.compile(r'^\w{1,6} (.*) \(')
            model_n = model_r.findall(processor)
            processor_model.extend(model_n)

            core_r = re.compile(r'\((\d{1,2}) ')
            core = core_r.findall(processor)
            cores.append(int(core[0]))

            thread_r = re.compile(r'\(\d{1,2} núcleos \/ (\d{1,2}) ')
            thread = thread_r.findall(processor)
            threads.append(int(thread[0]))


            mhz_r = re.compile(r'\(\d{1,2} núcleos \/ \d{1,2} hilos \/ (.*)\)')
            mhz = mhz_r.findall(processor)
            processor_mhz.extend(mhz)

    return processor_brand,processor_model,cores,threads,processor_mhz

def details_graphics(graphics):
    graphic_integrated = []
    dedicated_graphic = []
    
    for graphic in graphics:
        integrated_r = re.compile(r'^(.*) \(Integrada\)|\(128 MB\)')
        integrated = integrated_r.findall(graphic)
        if len(integrated) == 0:
            graphic_integrated.append('No tiene')
        else:
            graphic_integrated.extend(integrated)

        have_integrated_r = re.compile(r'\(Integrada\)|\(128 MB\)')
        #El 128 es por las Irish Graphics de los Mac que no se indican como integradas, pese a que lo son
        #pero todas tienen (128 MB) al final 
        if have_integrated_r.search(graphic) == None:
                dedicated_graphic.append(graphic)
        else:
            dedicated_r = re.compile(r'\(Integrada\)(.+? \(\d{1,2} GB\))$')
            dedicated = dedicated_r.findall(graphic)
            if len(dedicated) == 0:
                dedicated_graphic.append('No tiene')
            else:
                dedicated_graphic.extend(dedicated)
        
    return graphic_integrated,dedicated_graphic

In [ ]:
def extract_convert():
    '''
    Funcion que retorna un dataframe con los datos obtenidos del scrapeo de Notebooks en SoloTodo,
    ya filtrados y depurados
    '''
    product_url_base = 'https://www.solotodo.cl/notebooks'
    
    url_list = create_urls(product_url_base)
    
    #para hacer pruebas solamente
    #url_list =  ['https://www.solotodo.cl/notebooks?ordering=offer_price_usd&page=1&',
    #            'https://www.solotodo.cl/notebooks?ordering=offer_price_usd&page=2&',
    #             'https://www.solotodo.cl/notebooks?ordering=offer_price_usd&page=3&',
    #             'https://www.solotodo.cl/notebooks?ordering=offer_price_usd&page=4&',
    #             'https://www.solotodo.cl/notebooks?ordering=offer_price_usd&page=5&']
    
    
    #Extracción en bruto
    names,prices,processor,ram,screen,storages,graphics,links,ids,Brand = extract_info(url_list)
    
    #Filtrado de datos
    eMMC,HDD,SSD,Optane = details_storage(storages)
    Inches,Resolution,Hz = details_screen(screen)
    Ram_gb,DDR,Mhz = details_ram(ram)
    processor_brand,processor_model,cores,threads,processor_mhz = details_processor(processor)
    graphic_integrated,dedicated_graphic = details_graphics(graphics)
    
    #Armado del Dataframe
    today = datetime.date.today().strftime('%d-%m-%Y')
    info = {'Id':ids ,
            'link':links,
            'Name':names , 
            'Brand':Brand,
            'Category':'Notebook',
            'Processor brand': processor_brand,
            'Processor Model': processor_model,
            'Cores': cores,
            'Threads':threads,
            'Processor mhz':processor_mhz,
            'Ram (GB)':Ram_gb, 
            'Ram (DDR)': DDR,
            'Ram (Mhz)': Mhz,
            'Screen (Resolution)':Resolution,
            'Screen (Inches)':Inches,
            'Screen (Hz)':Hz,
            'eMMC (GB)':eMMC,
            'HDD (GB)':HDD,
            'SSD (GB)':SSD,
            'Intel Optane (GB)':Optane,
            'Graphic Integrated':graphic_integrated, 
            'Graphic Dedicated':dedicated_graphic,
            'Date':today,
            'Prices' :prices}
    
    df = pd.DataFrame(info)
    return df


# Execution

In [ ]:
today = datetime.date.today().strftime('%d-%m-%Y')
dir = '/content/drive/My Drive/Notebooks Dataset/{}.csv'.format(today)
notebooks_df = extract_convert()
notebooks_df.to_csv(dir, index=False)

#To do: cambiar dato entregado como fecha

Hay que extraer info de 54 paginas
Se han creado los 54 links para comenzar el Scrapping
Se extá obteniendo la información broda, esto demorará 324 segundos 
Lista la hoja numero : 1
Lista la hoja numero : 2
Lista la hoja numero : 3
Lista la hoja numero : 4
Lista la hoja numero : 5
Lista la hoja numero : 6
Lista la hoja numero : 7
Lista la hoja numero : 8
Lista la hoja numero : 9
Lista la hoja numero : 10
Lista la hoja numero : 11
Lista la hoja numero : 12
Lista la hoja numero : 13
Lista la hoja numero : 14
Lista la hoja numero : 15
Lista la hoja numero : 16
Lista la hoja numero : 17
Lista la hoja numero : 18
Lista la hoja numero : 19
Lista la hoja numero : 20
Lista la hoja numero : 21
Lista la hoja numero : 22
Lista la hoja numero : 23
Lista la hoja numero : 24
Lista la hoja numero : 25
Lista la hoja numero : 26
Lista la hoja numero : 27
Lista la hoja numero : 28
Lista la hoja numero : 29
Lista la hoja numero : 30
Lista la hoja numero : 31
Lista la hoja numero : 32
Lista la hoja numer

#Output

In [ ]:
notebooks = pd.read_csv('01-08-2020.csv')
notebooks

,Id,link,Name,Brand,Category,Processor brand,Processor Model,Cores,Threads,Processor mhz,Ram (GB),Ram (DDR),Ram (Mhz),Screen (Resolution),Screen (Inches),Screen (Hz),eMMC (GB),HDD (GB),SSD (GB),Intel Optane (GB),Graphic Integrated,Graphic Dedicated,Date,Prices
0,80680-lenovo-ideapad-1-14igl05-81vu001dcl,https://www.solotodo.cl/products/80680-lenovo-...,Lenovo IdeaPad 1 14IGL05 [81VU001DCL],LENOVO,Notebook,Intel,Celeron N4020,2,2,1100 MHz - 2800 MHz,4,DDR4,2400,1366x768,14.0,60,64,0,0,0,Intel UHD Graphics 600,No tiene,01-08-2020,229990
1,84361-lenovo-ideapad-3-14-81w0001bcl,https://www.solotodo.cl/products/84361-lenovo-...,Lenovo IdeaPad 3 14 [81W0001BCL],LENOVO,Notebook,AMD,A4-3300M,2,2,1900 MHz - 2500 MHz,4,DDR4,2400,1920x1080,14.0,60,0,500,0,0,AMD Radeon HD 6480G,No tiene,01-08-2020,249990
2,84211-hp-240-g7-1d0f9lt,https://www.solotodo.cl/products/84211-hp-240-...,HP 240 G7 [1D0F9LT],HP,Notebook,Intel,Celeron N4020,2,2,1100 MHz - 2800 MHz,4,DDR4,2400,1366x768,14.0,60,0,500,0,0,Intel UHD Graphics 600,No tiene,01-08-2020,259670
3,80041-asus-x509ma-br112t-90nb0q32-m01660,https://www.solotodo.cl/products/80041-asus-x5...,ASUS X509MA-BR112T [90NB0Q32-M01660],ASUS,Notebook,Intel,Celeron N4000,2,2,1100 MHz - 2600 MHz,4,DDR4,2400,1366x768,15.6,60,0,500,0,0,Intel UHD Graphics 600,No tiene,01-08-2020,269990
4,64049-hp-14-cm0029la-6qv28la,https://www.solotodo.cl/products/64049-hp-14-c...,HP 14-CM0029LA [6QV28LA],HP,Notebook,AMD,A4-9125,2,4,2300 MHz - 2600 MHz,4,DDR4,1866,1366x768,14.0,60,0,500,0,0,AMD Radeon R3 Graphics (Mullins/Beema),No tiene,01-08-2020,269990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,68193-asus-zenbook-pro-15-ux581gv-h2001t-90nb0...,https://www.solotodo.cl/products/68193-asus-ze...,ASUS Zenbook Pro 15 UX581GV-H2001T [90NB0NG1-M...,ASUS,Notebook,Intel,Core i9-9980HK,8,16,2400 MHz - 5000 MHz,32,DDR4,2666,3840x2160,15.6,60,0,0,1000,0,Intel UHD Graphics 630,NVIDIA GeForce RTX 2060 (6 GB),01-08-2020,2899990
635,62687-msi-ws65-9tk-512-ssd,https://www.solotodo.cl/products/62687-msi-ws6...,MSI WS65 9TK (512 SSD),MSI,Notebook,Intel,Core i9-9880H,8,16,2300 MHz - 4800 MHz,32,DDR4,2666,1920x1080,15.6,60,0,0,512,0,Intel UHD Graphics 630,NVIDIA Quadro RTX 3000 (6 GB),01-08-2020,3006990
636,69645-hp-omen-x-15-dg0001la-6qv31la,https://www.solotodo.cl/products/69645-hp-omen...,HP Omen X 15-DG0001LA [6QV31LA],HP,Notebook,Intel,Core i9-9880H,8,16,2300 MHz - 4800 MHz,32,DDR4,2666,1920x1080,15.6,240,0,0,512,0,Intel UHD Graphics 630,NVIDIA GeForce RTX 2080 Max-Q (8 GB),01-08-2020,3509990
637,83099-asus-rog-zephyrus-duo-gx550lxs-hc106t-90...,https://www.solotodo.cl/products/83099-asus-ro...,ASUS ROG Zephyrus Duo GX550LXS-HC106T [90NR02Z...,ASUS,Notebook,Intel,Core i9-10980HK,8,16,2400 MHz - 5100 MHz,32,DDR4,3200,3840x2160,15.6,60,0,0,2000,0,Intel UHD Graphics 630,NVIDIA GeForce RTX 2080 Super Max-Q (8 GB),01-08-2020,3999990


In [ ]:
notebooks.dtypes

Id                      object
link                    object
Name                    object
Brand                   object
Category                object
Processor brand         object
Processor Model         object
Cores                    int64
Threads                  int64
Processor mhz           object
Ram (GB)                 int64
Ram (DDR)               object
Ram (Mhz)                int64
Screen (Resolution)     object
Screen (Inches)        float64
Screen (Hz)              int64
eMMC (GB)                int64
HDD (GB)                 int64
SSD (GB)                 int64
Intel Optane (GB)        int64
Graphic Integrated      object
Graphic Dedicated       object
Date                    object
Prices                   int64
dtype: object

In [ ]:
Today = datetime.date.today().strftime('%d-%m-%Y %H:%M:%S %p')
notebooks['Date'] = Today

In [ ]:
notebooks['Date'] = pd.to_datetime(
notebooks['Date'],
errors = 'coerce',
format ='%d-%m-%Y %H:%M:%S %p'
)

In [ ]:
notebooks.dtypes

Id                             object
link                           object
Name                           object
Brand                          object
Category                       object
Processor brand                object
Processor Model                object
Cores                           int64
Threads                         int64
Processor mhz                  object
Ram (GB)                        int64
Ram (DDR)                      object
Ram (Mhz)                       int64
Screen (Resolution)            object
Screen (Inches)               float64
Screen (Hz)                     int64
eMMC (GB)                       int64
HDD (GB)                        int64
SSD (GB)                        int64
Intel Optane (GB)               int64
Graphic Integrated             object
Graphic Dedicated              object
Date                   datetime64[ns]
Prices                          int64
dtype: object

In [ ]:
notebooks['Graphic Integrated'].isnull().nunique()

2

In [ ]:
notebooks['Graphic Integrated'] = notebooks['Graphic Integrated'].fillna('Intel Iris Plus Graphics 655 (128 MB)')
#La web tenia las graficas integradas de apple con un formato totalmente distinto al resto, por eso las 4 filas que les faltaba ese dato se les ingresa a mano.

In [ ]:
notebooks['Graphic Integrated'].isnull().nunique()

1

In [ ]:
# Dataframe listo
notebooks.to_csv('/content/drive/My Drive/Notebooks Dataset/Notebooks_df.csv', index=False)




### Clipboard

In [ ]:
#API para obtener el precio del dolar en el dia, se usará más adelante para generar una versión del dataset en dolares.
def get_dolar_value():
    today = datetime.date.today().strftime('%d-%m-%Y')
    url = 'https://mindicador.cl/api/dolar/{}'.format(today)
    response = requests.get(url)
    data = json.loads(response.text.encode("utf-8"))
    dolar = data['serie'][0]['valor']
    return dolar
dolar = get_dolar_value()
dolar
